In [2]:
"""
10-meter aspen patch metrics
author: maxwell.cook@colorado.edu
"""

import os, sys, time
import geopandas as gpd
import pylandstats as pls
import multiprocessing as mp
import concurrent.futures
import numpy as np
import rasterio as rio
from rasterio.mask import mask
from multiprocessing import Pool, cpu_count
from tqdm.notebook import tqdm
from shapely.geometry import box

maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/'
projdir = os.path.join(maindir, 'aspen-fire/')

# Custom functions
sys.path.append(os.path.join(maindir,'aspen-fire/Aim2/code/Python'))
from __functions import *

proj = 'EPSG:5070' # albers

print("Ready to go !")

Ready to go !


In [3]:
fp = os.path.join(projdir, 'Aim3/data/spatial/raw/fsim/firesheds/subfiresheds.gpkg')
firesheds = gpd.read_file(fp)
firesheds.columns

Index(['OBJECTID', 'Fireshed_ID', 'Subfireshed_ID', 'Fireshed_Area_Ha',
       'Subfireshed_Area_Ha', 'Fireshed_Name', 'Fireshed_Code',
       'Fireshed_State', 'Shape_Length', 'Shape_Area', 'Fireshed_MajRegion',
       'AnnualExposure', 'PctRecentlyDisturbed', 'geometry'],
      dtype='object')

In [4]:
firesheds = firesheds[['Subfireshed_ID','Subfireshed_Area_Ha','geometry']]
firesheds.rename(columns={
    'Subfireshed_ID': 'sfs_id',
    'Subfireshed_Area_Ha': 'sfs_area_ha'
}, inplace=True)
print(len(firesheds))
firesheds.columns

1714


Index(['sfs_id', 'sfs_area_ha', 'geometry'], dtype='object')

In [5]:
firesheds['sfs_area_ha'].describe()

count     1714.000000
mean     10317.604201
std       1255.129538
min       5832.000000
25%       9564.480000
50%      10380.960000
75%      11197.440000
max      14580.000000
Name: sfs_area_ha, dtype: float64

In [6]:
# patch analysis - largest patch size, mean patch size, etc
dir = os.path.join(projdir,'Aim1/data/spatial/mod/results/classification/')
aspen10_fp = os.path.join(dir,'s2aspen_distribution_10m_y2019_CookEtAl.tif')

# Define metrics to calculate
cls_metrics = ['number_of_patches', 'patch_density', 'largest_patch_index']

# Function to process a single fireshed grid
def process_fireshed(fireshed):
    """ Process a single fireshed grid for patch statistics. """
    try:
        unit_id = fireshed["sfs_id"]
        geometry = fireshed["geometry"]  # Shapely Polygon

        with rio.open(aspen10_fp) as src:
            # Extract raster subset using rasterio mask (faster than .rio.clip)
            out_image, _ = mask(src, [geometry], crop=True)
            arr = out_image.squeeze()

        # Ensure valid data exists
        if np.all(arr == src.nodata) or np.count_nonzero(arr) == 0:
            return None

        # Compute patch statistics
        ls = pls.Landscape(arr, res=(10, 10))
        patches = ls.compute_class_metrics_df(metrics=cls_metrics)
        # retain only aspen patches
        patches = patches.reset_index() # set "class_val"
        patches = patches[patches["class_val"] == 1].copy()
        if not patches.empty:
            patches["sfs_id"] = unit_id
            return patches
        else:
            return None

    except Exception as e:
        print(f"Error processing grid {fireshed['sfs_id']}: {e}")
        return None

if __name__ == "__main__":
    t0 = time.time()

    num_workers = 2  # adjust as needed
    fs_list = firesheds.to_dict(orient="records")  # Convert GeoDataFrame to list

    results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
        futures = {executor.submit(process_fireshed, fs): fs for fs in fs_list}
        
        for future in tqdm(concurrent.futures.as_completed(futures), 
                           total=len(futures), desc="Processing firesheds"):
            result = future.result()
            if result is not None:
                results.append(result)

    # Merge results into a DataFrame
    patch_metrics_df = pd.concat(results, ignore_index=True) if results else pd.DataFrame()

    t1 = (time.time() - t0) / 60
    print(f"\nTotal elapsed time: {t1:.2f} minutes.\n")

Processing firesheds:   0%|          | 0/1714 [00:00<?, ?it/s]


Total elapsed time: 1.24 minutes.



In [7]:
patch_metrics_df.head()

,class_val,number_of_patches,patch_density,largest_patch_index,sfs_id
0,1,86,3.815372,0.095384,20467
1,1,4,0.161778,0.010920,20466
2,1,1941,54.625124,2.078062,20469
3,1,1147,34.018549,2.622127,21247
4,1,274,6.027091,0.690916,21025


In [8]:
len(patch_metrics_df)

1635

In [9]:
# save this file out.
out_fp = os.path.join(projdir,'Aim3/data/tabular/firesheds_aspen10_patches.csv')
patch_metrics_df.to_csv(out_fp)
print(f"Saved to: {out_fp}")

Saved to: /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim3/data/tabular/firesheds_aspen10_patches.csv
